# Notebook pour Devoir 4 Question programmation

### Objectifs d'apprentissage
Dans ce problème, nous allons expérimenter avec des modèles de langage (LMs) et implémenter le lissage. Nous verrons également les effets de l'utilisation des LM bigrammes.

### Rédaction du code
Cherchez le mot-clé "TODO" et remplissez votre code dans l'espace vide.
N'hésitez pas à ajouter et à supprimer des arguments dans les signatures de fonctions, mais faites attention au fait que vous pourriez avoir besoin de les modifier dans des appels de fonctions qui sont déjà présents dans le notebook.


### Prétraitement des données

Dans cette section, nous allons écrire du code pour charger les données et les nettoyer (tokenize).

In [ ]:
# Importer des bibliothèques pour le prétraitement
import os
import numpy as np
import nltk
nltk.download('punkt')

[Natural Language Toolkit](https://www.nltk.org/)

NLTK est une plateforme de premier plan pour la création de programmes Python destinés à travailler avec des données sur le langage humain. Elle fournit des interfaces faciles à utiliser pour plus de 50 corpus et ressources lexicales telles que WordNet, ainsi qu'une suite de bibliothèques de traitement de texte pour la classification, la tokénisation, le stemming, l'étiquetage, l'analyse syntaxique et le raisonnement sémantique, des wrappers pour des bibliothèques NLP industrielles, et un forum de discussion actif.

In [ ]:
# Classe Tokenizer
# Remplir tous les blocs fonctionnels marqués comme TODO

class Tokenizer:
  def __init__(self, tokenize_type='basic', lowercase=False):
    self.lowercase = lowercase  # If this is True, convert text to lowercase while tokenizing.
    self.type = tokenize_type
    self.vocab = []
    

  """This simple tokenizer splits the text using whitespace."""
  def basicTokenize(self, string):
    words = string.split()
    return words

  ### TODO : Complétez cette fonction pour utiliser la fonction word_tokenize() de NLTK. ###
  ### renvoie aussi une liste de mots similaire à la fonction basicTokenize.
  def nltkTokenize(self, string):
    ##### DEBUT SOLUTION #####
     
    ##### FIN SOLUTION #####
    

  def tokenize(self, string):
    if self.lowercase:
      string = string.lower()
    if self.type == 'basic':
      tokens = self.basicTokenize(string)
    elif self.type == 'nltk':
      tokens = self.nltkTokenize(string)
    else:
      raise ValueError('Unknown tokenization type.')    


    # Remplissage du vocabulaire
    self.vocab += [w for w in set(tokens) if w not in self.vocab]

    return tokens

  ### TODO : Compléter cette fonction pour retourner les k premiers mots du corpus et leurs fréquences correspondantes. 
  ### Les k premiers mots sont triés dans le corpus selon leur fréquence. ###
  ### retourne une liste
  def countTopWords(self, words, k):
    ##### DEBUT SOLUTION #####

    ##### FIN SOLUTION ##### 
    

In [ ]:
# Fonction pour la lecture du corpus
def readCorpus(filename, tokenizer):  
  with open(filename) as f:
    words = tokenizer.tokenize(f.read())
  return words

### Modélisation du langage et lissage 
Dans cette section, nous allons d'abord calculer le nombre de bigrammes et estimer les probabilités de bigrammes. Nous mettrons ensuite en œuvre le lissage de Laplace add-alpha (également appelé add-k) pour modifier les probabilités. En classe, nous avons vu un cas particulier de lissage add-alpha qui est le lissage add-1 où alpha est égal à 1 ($\alpha = 1$). Dans ce devoir, nous allons essayer différentes valeurs de alpha pour le lissage.
L'estimation d'une probabilité conditionnelle bigramme du mot suivant $w_n$ étant donné le mot préfixe $w_{n-1}$ en utilisant le lissage add-alpha s'exprime comme suit :



<h1><center>$\hat{P}_{add-\alpha}(w_n|w_{n-1}) = \frac{C(w_{n-1}w_n)+\alpha}{C(w_{n-1})+\alpha|V|}$</center></h1>


In [ ]:
# Importer des bibliothèques
# N'hésitez pas à en importer autant que vous le souhaitez.
import sys
import os
import numpy as np
import argparse
from tqdm import tqdm
from collections import Counter

In [ ]:
# Définition de la classe pour la modélisation du langage
# Remplir tous les blocs fonctionnels marqués comme TODO.

class LanguageModel:
  def __init__(self, vocab, n=2, smoothing=None, smoothing_param=None):
    assert n >=2, "This code does not allow you to train unigram models."
    self.vocab = vocab
    self.token_to_idx = {word: i for i, word in enumerate(self.vocab)}
    self.n = n
    self.smoothing = smoothing    
    self.smoothing_param = smoothing_param
    self.x = None      # Contient le compte de bigramme
    self.bi_prob = None        # Contient les probabilités de bigramme calculées.

    assert smoothing is None or smoothing_param is not None, "Forgot to specify a smoothing parameter?"


  """Calculer les probabilités de bigramme de base (sans aucun lissage)"""
  def computeBigramProb(self):
    self.bi_prob = self.bi_counts.copy()

    for i, _ in enumerate(tqdm(self.bi_prob, desc="Estimating bigram probabilities")):
      cnt = np.sum(self.bi_prob[i])
      if cnt > 0:
        self.bi_prob[i] /= cnt
        
  ### TODO : calculer les probabilités de bigramme avec le lissage Add-alpha.###
  ### Vous pouvez suivre la même structure que la fonction ci-dessus computeBigramProb(self)
  ### Pour une implémentation efficace, essayez de vectoriser autant que possible et évitez les boucles for imbriquées.
  ### Il n'est pas nécessaire de retourner quelque chose ici
  def computeBigramProbAddAlpha(self, alpha=0.001):
    ##### DEBUT SOLUTION #####
     
    ##### FIN SOLUTION #####
    return


  """Entraîner un modèle de langage n-gram de base"""
  def train(self, corpus):
    if self.n==2:
      self.bi_counts = np.zeros((len(self.vocab), len(self.vocab)), dtype=float)
    else:
      raise ValueError("Only bigram model has been implemented so far.")
    
    # Convertir en indices de tokens/jetons.
    corpus = [self.token_to_idx[w] for w in corpus]

    # Rassembler les comptes
    for i, idx in enumerate(tqdm(corpus[:1-self.n], desc="Counting")):
      self.bi_counts[idx][corpus[i+1]] += 1

    # Pré-calculer les probabilités.
    if not self.smoothing:
      self.computeBigramProb()
    elif self.smoothing == 'addAlpha':
      self.computeBigramProbAddAlpha(self.smoothing_param)
    else:
      raise ValueError("Unknown smoothing type.")



  def test(self, corpus):
    
    logprob = 0.

    # Convertir en indices de tokens/jetons.
    corpus = [self.token_to_idx[w] for w in corpus]

    for i, idx in enumerate(tqdm(corpus[:1-self.n], desc="Evaluating")):
      logprob += np.log(self.bi_prob[idx, corpus[i+1]])

    logprob /= len(corpus[:1-self.n])

    # Calculer la perplexité
    ppl = np.exp(-logprob)

    return ppl

### Instanciation d'un tokenizer et d'un LM, et calcul de la perplexité
Cette section contient le code du pilote pour l'apprentissage d'un modèle de langue et son évaluation sur un corpus de train et de dev.

In [ ]:
def runLanguageModel(train_corpus,
                     val_corpus,
                     train_fraction,
                     tokenizer,
                     smoothing_type=None,
                     smoothing_param='0.0'):

  # Instanciation du modèle de langage.
  lm = LanguageModel(tokenizer.vocab, n=2, smoothing=smoothing_type, smoothing_param=smoothing_param)

  # Déterminer l'indice pour un pourcentage spécifique du corpus d'entraînement à utiliser.
  train_idx = int(train_fraction * len(train_corpus))

  lm.train(train_corpus[:train_idx])

  train_ppl = lm.test(train_corpus[:train_idx])
  val_ppl = lm.test(val_corpus)

  print("Train perplexity: %f, Val Perplexity: %f" %(train_ppl, val_ppl))

  return [train_ppl, val_ppl]

### Charger les données

In [ ]:
#Monter drive pour accéder aux fichiers dans gdrive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [ ]:
#Vous devez changer le chemin du fichier train et du fichier val en fonction de l'endroit où vous les avez stockés dans votre gdrive.

### TODO : changer le chemin du fichier train_file
train_file = '/content/gdrive/MyDrive/Assignment 3/brown-train.txt'
### TODO : changer le chemin pour val_file
val_file = '/content/gdrive/MyDrive/Assignment 3/brown-val.txt'

# Instanciation d'un tokenizer de base
basic_tokenizer = Tokenizer(tokenize_type='basic', lowercase=True)

# Lire le corpus et le stocker
train_corpus = readCorpus(train_file, basic_tokenizer)
val_corpus = readCorpus(val_file, basic_tokenizer)

# Instanciation du tokenizer nltk
nltk_tokenizer = Tokenizer(tokenize_type='nltk', lowercase=True)

# Lire le corpus et le stocker
train_corpus_nltk = readCorpus(train_file, nltk_tokenizer)
val_corpus_nltk = readCorpus(val_file, nltk_tokenizer)

#### Partie du code pour la sous-partie (a)
Imprimez les 10 premiers mots les plus fréquents

In [ ]:
### TODO : Imprimez les 10 premiers (plus fréquents) mots du corpus à la fois en utilisant un tokenizer basique et en utilisant nltk tokenizer. 
### IMPORTANT : complétez d'abord la fonction countTopWords dans la classe Tokenizer.
### Utilisez une syntaxe similaire à celle-ci : print("Top 10 words basic : %s" % A REMPLIR)

##### DEBUT SOLUTION #####

##### FIN SOLUTION #####

### Experiences

#### Tracer la fréquence des mots
Code pour la sous-partie (b)

En utilisant la fonction nltkTokenize que vous avez écrite, faites un graphique des fréquences des mots dans le corpus d'entraînement, ordonné par leur rang, c'est-à-dire le mot le plus fréquent en premier, le deuxième mot le plus fréquent ensuite, et ainsi de suite sur l'axe des x.

In [ ]:
### TODO : Code pour tracer la fréquence des mots.
##### DEBUT SOLUTION #####

##### FIN SOLUTION #####

#### Rapport sur la perplexité de l'apprentissage et du test après l'apprentissage du modèle de langage
Code pour la sous-partie (c)

Utilisez la fonction basicTokenize et le modèle de langage bigramme ($n = 2$) sans lissage pour cette question.

Entraînez le modèle de langage et rapportez sa perplexité sur les ensembles d'entraînement et de validation.

In [ ]:
### TODO : Entraîner le modèle de langage bigramme sur l'ensemble du corpus d'entraînement (train fraction - fraction d'entraînement = 1) 
### et évaluer la perplexité à la fois sur le corpus d'entraînement et de validation.
##### DEBUT SOLUTION #####

##### FIN SOLUTION #####

#### Lissage Add-alpha
Code pour la sous-partie (d)

Utilisez la fonction basicTokenize et le modèle de langage bigramme ($n = 2$) avec lissage pour cette question.

Implémentez le lissage de Laplace (add-$\alpha$) dans la fonction appropriée fournie (computeBigramAddAlpha dans la classe LanguageModel) et entraînez le modèle avec le lissage add-alpha sur l'ensemble de l'entraînement pour différentes valeurs alpha $[10^{-5},10^{-4},10^{-3},10^{-2},10^{-1},1,1.5,2]$.

In [ ]:
### TODO : Pour différentes valeurs de alpha, entraîner le modèle de langage bigramme avec lissage sur le corpus d'entraînement entier (train fraction - fraction d'entraînement = 1).
### et évaluer la perplexité sur le corpus d'entraînement et de validation.
### Tracez la perplexité sur les ensembles de formation et de validation en fonction de alpha.
### Valeurs de alpha spécifiées ci-dessus
##### DEBUT SOLUTION #####

###### FIN SOLUTION #####